In [1]:
from gensim.models.fasttext import load_facebook_model
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd
import numpy as np
import gc

In [2]:
csv_path = '../annotations/words/all_words.csv'

df = pd.read_csv(csv_path)

df.head()

,word,sentiment
0,wszystkie,Neutral
1,luzacki,Positive
2,nudne,Negative
3,wporzadku,Positive
4,groźny,Negative


In [3]:
words = df['word'].tolist()

In [4]:
model_path = 'fasttext/cc.pl.300.bin'

model = load_facebook_model(model_path)

In [5]:
def get_embedding(word):
    if word in model.wv:
        return model.wv[word]
    else:
        return [0.0] * model.vector_size

print("Pobieranie embeddingów dla słów...")
embeddings = df['word'].apply(get_embedding).tolist()
embeddings_matrix = np.array(embeddings)
df['embedding'] = embeddings

df.head()

Pobieranie embeddingów dla słów...


,word,sentiment,embedding
0,wszystkie,Neutral,"[0.04213148, 0.03196649, 0.014593786, 0.007251..."
1,luzacki,Positive,"[-0.00025074184, 0.060088143, -0.06687481, -0...."
2,nudne,Negative,"[-0.04458306, 0.045463443, -0.005535635, -0.00..."
3,wporzadku,Positive,"[-0.018328566, 0.05708965, 0.018148778, 0.0063..."
4,groźny,Negative,"[-0.02980628, 0.008906932, -0.03944085, -0.072..."


In [6]:
k = 5

def get_top_k_similar(word, k):
    if word in model.wv:
        similar = model.wv.most_similar(word, topn=k)
        return [sim_word for sim_word, score in similar]
    else:
        return [None] * k  # Jeśli słowo nie jest w słowniku


for i in range(1, k+1):
    df[f'similar_word_{i}'] = df['word'].apply(lambda x: get_top_k_similar(x, k)[i-1])

df.head()

,word,sentiment,embedding,similar_word_1,similar_word_2,similar_word_3,similar_word_4,similar_word_5
0,wszystkie,Neutral,"[0.04213148, 0.03196649, 0.014593786, 0.007251...",wszytskie,wszelkie,pozostałe,Wszystkie,wszyskie
1,luzacki,Positive,"[-0.00025074184, 0.060088143, -0.06687481, -0....",nonszalancki,luzackie,luzacko,zawadiacki,wyluzowany
2,nudne,Negative,"[-0.04458306, 0.045463443, -0.005535635, -0.00...",monotonne,nudnawe,Nudne,nużące,nudne.
3,wporzadku,Positive,"[-0.018328566, 0.05708965, 0.018148778, 0.0063...",wporządku,wpozadku,porzadku,porzadku.,pozadku
4,groźny,Negative,"[-0.02980628, 0.008906932, -0.03944085, -0.072...",niebezpieczny,niegroźny,groźniejszy,najgroźniejszy,szkodliwy


In [7]:
print("Zwalnianie pamięci zajmowanej przez model FastText...")
del model
gc.collect()
print("Pamięć zwolniona.")

Zwalnianie pamięci zajmowanej przez model FastText...
Pamięć zwolniona.


In [8]:
df.to_csv('word-fasttext-embeddings.csv', index=False)

In [9]:
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42, init='random')
tsne_result = tsne.fit_transform(embeddings_matrix)
df['tsne_x'] = tsne_result[:, 0]
df['tsne_y'] = tsne_result[:, 1]

In [10]:
sentiment_colors = {'Negative': 'red', 'Positive': 'green', 'Neutral': 'blue'}

In [11]:
fig_tsne = px.scatter(
    df,
    x='tsne_x',
    y='tsne_y',
    color='sentiment',
    title='Wizualizacja embeddingów słów za pomocą t-SNE',
    labels={'tsne_x': 't-SNE 1', 'tsne_y': 't-SNE 2'},
    hover_data=['word', 'sentiment'],
    color_discrete_map=sentiment_colors
)

fig_tsne.update_traces(textposition='top center')

fig_tsne.show()

In [13]:
df.head(10)

,word,sentiment,embedding,similar_word_1,similar_word_2,similar_word_3,similar_word_4,similar_word_5,tsne_x,tsne_y
0,wszystkie,Neutral,"[0.04213148, 0.03196649, 0.014593786, 0.007251...",wszytskie,wszelkie,pozostałe,Wszystkie,wszyskie,-6.951481,8.815587
1,luzacki,Positive,"[-0.00025074184, 0.060088143, -0.06687481, -0....",nonszalancki,luzackie,luzacko,zawadiacki,wyluzowany,5.223829,9.193687
2,nudne,Negative,"[-0.04458306, 0.045463443, -0.005535635, -0.00...",monotonne,nudnawe,Nudne,nużące,nudne.,-13.633554,9.173189
3,wporzadku,Positive,"[-0.018328566, 0.05708965, 0.018148778, 0.0063...",wporządku,wpozadku,porzadku,porzadku.,pozadku,-0.277524,8.035762
4,groźny,Negative,"[-0.02980628, 0.008906932, -0.03944085, -0.072...",niebezpieczny,niegroźny,groźniejszy,najgroźniejszy,szkodliwy,9.015926,9.217252
5,mieszane,Neutral,"[-0.04262481, 0.019884935, 0.0288956, 0.035826...",Mieszane,rozmieszane,ambiwalentne,przemieszane,zmieszane,-5.652344,6.266264
6,fajny,Positive,"[-0.04934426, 0.0101640625, 0.027789254, -0.04...",świetny,ciekawy,ładny,niezły,Fajny,10.134853,15.341961
7,dobra,Positive,"[0.024816966, 0.04614558, -0.009925917, -0.011...",",dobra",Dobra,zła,b.dobra,dobra-,16.301083,-6.064150
8,duza,Neutral,"[-0.025002176, -0.020540046, 0.020168146, 0.01...",mala,duża,wieksza,nieduza,najwieksza,16.000113,-11.854917
9,totalny,Neutral,"[-0.003404307, 0.063702695, 0.011909514, 0.016...",Totalny,zupełny,absolutny,istny,masakryczny,9.406193,7.015164
